In [1]:
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import os
from time import sleep
import time
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import random 

In [2]:
import spacy
nlp = spacy.load("pt_core_news_sm")

c:\Users\r-omd\.conda\envs\tf-venv39\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Users\r-omd\.conda\envs\tf-venv39\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]
c:\Users\r-omd\.conda\envs\tf-venv39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
base_inicial = "links_folha_04_2024.csv"
# base_inicial = "links_estadao_04_2024.csv"
# base_inicial = "links_correio_04_2024.csv"

df = pd.read_csv(base_inicial)
df = df.reset_index()
df.columns = ["n","links"]
df.shape


(110, 2)

In [49]:
df.head()

,n,links
0,0,https://www1.folha.uol.com.br/blogs/saude-ment...
1,1,https://www1.folha.uol.com.br/cotidiano/2024/0...
2,2,https://www1.folha.uol.com.br/equilibrioesaude...
3,3,https://www1.folha.uol.com.br/ciencia/2023/12/...
4,4,https://www1.folha.uol.com.br/equilibrioesaude...


In [50]:
def limpa_paragrafos(link, tamanho):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
    try:
        req = Request(link, headers = headers)
        response = urlopen(req)
       
    except HTTPError as e: # erro de segurança do site, pode ser necessário usar user-Agent
        print(e.status, e.reason)
        return ["404_nao_encontrado"]

    except URLError as e:
        print(e.reason)
        return ["404_nao_encontrado"]

    t = random.randint(1, 10)
    # time.sleep(t)
    
#     html = urllib.request.urlopen(link).read()
    html = response.read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)
    
#     print(lista)

    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))
#             print(miolo)

    paragrafos = []
    for i in range(len(miolo)):
        if 'esquizo' in miolo[i]:
            paragrafos.append(miolo[i])
    
    t = random.randint(1, 10)
    time.sleep(t)
    
#     for i in range(len(paragrafos)):
#         lista_final = []
#         doc = nlp(paragrafos[i].lower())
#         for token in doc:
#             valido = not token.is_stop and token.is_alpha
# #             valido = token.is_stop
#             if valido:
#                 lista_final.append(token.text)
    
#         limpo = ' '.join(lista_final)
#         paragrafos[i] = limpo
#         time.sleep(1)

    return paragrafos, miolo

In [51]:
link = df.links.values[0]
link

'https://www1.folha.uol.com.br/blogs/saude-mental/2024/02/entenda-o-que-e-quadro-psicotico-agudo-que-vanessa-lopes-teve-no-bbb24.shtml'

In [52]:
paragrafos, miolo = limpa_paragrafos(link, 70)

In [53]:
str(paragrafos)

'[\'Durante o curso do episódio, afirma o médico, é importante o uso de medicamentos antipsicóticos para estabilização do quadro. "É importante destacar que muitos casos podem evoluir para diagnósticos definitivos em psiquiatria e necessitar de tratamento crônico, como no caso da esquizofrenia e do transtorno bipolar."\']'

In [54]:
str(miolo)

'[\'BBB: Entenda o que é quadro psicótico agudo - 18/02/2024 - Saúde Mental - Folha\', \'Sílvia Haidar é jornalista. Blog traz informações, serviços e entrevistas com especialistas sobre saúde mental.\', \'Entenda o que é quadro psicótico agudo, que Vanessa Lopes teve no BBB24\', \'Episódio é uma emergência psiquiátrica que pode ser causada por transtornos, fatores orgânicos, quadros infecciosos e traumas cerebrais, explica psiquiatra\', \', 22, participante do BBB24, desistiu do programa após apresentar confusão mental.\', \'Neste domingo (18), ela conta no programa Fantástico, da Globo, que teve um quadro psicótico agudo.\', \'O psiquiatra Adilon Harley Machado, da clínica Espaço Arquétipo, explica que o episódio psicótico agudo é uma emergência psiquiátrica caracterizada por alteração da percepção da realidade, desorganização do pensamento e do conteúdo do discurso, podendo se manifestar com delírios e alucinações, agitação psicomotora e perda do juízo crítico.\', \'Vanessa Lopes du

In [55]:
df.links.iloc[1]

'https://www1.folha.uol.com.br/cotidiano/2024/02/justica-da-holanda-manda-brasileiro-preso-com-carne-na-mala-para-tratamento-psiquiatrico.shtml'

In [56]:
n_links = df.shape[0]

In [57]:
for i in range(0, n_links):

    print(f"Processando {i} de {n_links}...")
    
    link = df.links.iloc[i]
    try: 
        paragrafos, miolo = limpa_paragrafos(link, 70)
        df.loc[(df.links == link), "paragrafos_resgatados"] = str(paragrafos)
        df.loc[(df.links == link), "texto_completo"] = str(miolo)
    except:
        df.loc[(df.links == link), "paragrafos_resgatados"] = "[]"
        df.loc[(df.links == link), "texto_completo"] = "[]"

Processando 0 de 110...
Processando 1 de 110...
Processando 2 de 110...
Processando 3 de 110...
Processando 4 de 110...
Processando 5 de 110...
Processando 6 de 110...
Processando 7 de 110...
Processando 8 de 110...
Processando 9 de 110...
Processando 10 de 110...
Processando 11 de 110...
Processando 12 de 110...
Processando 13 de 110...
Processando 14 de 110...
Processando 15 de 110...
Processando 16 de 110...
Processando 17 de 110...
Processando 18 de 110...
Processando 19 de 110...
Processando 20 de 110...
Processando 21 de 110...
Processando 22 de 110...
Processando 23 de 110...
Processando 24 de 110...
Processando 25 de 110...
Processando 26 de 110...
Processando 27 de 110...
Processando 28 de 110...
Processando 29 de 110...
Processando 30 de 110...
Processando 31 de 110...
Processando 32 de 110...
Processando 33 de 110...
Processando 34 de 110...
Processando 35 de 110...
Processando 36 de 110...
Processando 37 de 110...
Processando 38 de 110...
Processando 39 de 110...
Processand

In [59]:
df.head()

,n,links,paragrafos_resgatados,texto_completo
0,0,https://www1.folha.uol.com.br/blogs/saude-ment...,"['Durante o curso do episódio, afirma o médico...",['BBB: Entenda o que é quadro psicótico agudo ...
1,1,https://www1.folha.uol.com.br/cotidiano/2024/0...,['Embora tenham reconhecido que o réu agiu de ...,['Begoleã Mendes fará tratamento psiquiátrico ...
2,2,https://www1.folha.uol.com.br/equilibrioesaude...,"[', deficiência intelectual, esquizofrenia, al...",['Transtornos mentais afetam vida saudável de ...
3,3,https://www1.folha.uol.com.br/ciencia/2023/12/...,"['Em 2019, a equipe de van den Heuvel demonstr...",['A importância do tamanho do cérebro para a i...
4,4,https://www1.folha.uol.com.br/equilibrioesaude...,['Um outro estudo publicado pelo seu grupo em ...,['IA diagnostica autismo usando imagem da reti...


In [60]:
df1 = df.query("paragrafos_resgatados != '[]'")

In [61]:
df1.shape

(88, 4)

In [62]:
prox_pasta = "s1_pasta_04_2024"

In [63]:
df1.to_csv(f"../{prox_pasta}/{base_inicial}_completo_resgatado.csv", index = False)

In [64]:
# df[df.paragrafos_resgatados.notnull()].to_csv("links_estadao_resgatados_parte_3.csv", index = False)
# df[df.paragrafos_resgatados.isnull()].to_csv("links_estadao_nulos.csv", index = False)


In [65]:
# df = pd.read_csv("links_estadao_resgatados.csv")
# df.shape

In [66]:
t = df.query("paragrafos_resgatados == '[]'")
t

,n,links,paragrafos_resgatados,texto_completo
8,8,https://www1.folha.uol.com.br/colunas/monicabe...,[],['Fiocruz: Jovens sofrem um terço dos acidente...
9,9,https://www1.folha.uol.com.br/ciencia/2023/12/...,[],['Genes que aumentam a fertilidade também encu...
15,15,https://www1.folha.uol.com.br/equilibrio/2023/...,[],['Doença mental na 3ª idade aumenta o risco de...
17,17,https://www1.folha.uol.com.br/equilibrioesaude...,[],['Por que psiquiatras começam a prescrever Oze...
18,18,https://www1.folha.uol.com.br/equilibrio/2023/...,[],['Saiba como fazer tratamento para depressão n...
20,20,https://www1.folha.uol.com.br/equilibrio/2023/...,[],['Saúde mental é uma prioridade para saúde púb...
23,23,https://www1.folha.uol.com.br/equilibrio/2023/...,[],['Suicídio: estudo associa tentativa a fatores...
25,25,https://www1.folha.uol.com.br/equilibrio/2023/...,[],['Rachel Aviv escreve sobre transtornos mentai...
28,28,https://www1.folha.uol.com.br/equilibrioesaude...,[],['Paralisia cerebral: Neuralink iniciará teste...
29,29,https://www1.folha.uol.com.br/blogs/saude-ment...,[],['USP oferece palestras gratuitas sobre saúde ...


In [81]:
t.links.values[9]

'https://www1.folha.uol.com.br/blogs/saude-mental/2023/09/ipq-portas-abertas-oferece-mais-de-150-palestras-e-atividades-gratuitas-sobre-saude-mental.shtml'

In [82]:
t.shape

(22, 4)

In [83]:
t.to_csv(f"{base_inicial}_resgatado_vazio.csv", index = False)